# Technical Analysis Indicator Tools

In this notebook, several technical analysis indicators tools will be explored. Technical Analysis is the study of collective market sentiment, as expressed in the buying and selling of assets. It is based on the idea that prices are determined by the interaction of supply and demand. A key assumption of technical analysis is that the efficient markets hypothesis (The idea that asset prices reflect all available information) does NOT hold. 

The analysis will use historical data to indicate markets that are overbought or oversold. A possible trading strategy is to buy when the markets are Oversold and sell when the market is Overbought. This is an example of a contrarian strategy. Contrarians believe markets get overbought or oversold because most investors tend to buy and sell at the wrong times. Therefore, it can be profitable to trade in the opposite direction.

It is recommended to use several tools when using a strategy that involves Technical Analysis. As a result, after creating three different indicator tools, the signals will be combined and a recommendation will be generated using majority voting. If 2 or more tools signal that a market is overbought or oversold, a recommendation of "BUY" or "SELL" will be made.

### Importing the Required Libraries

In [1]:
import eikon as ek
from eikon.tools import get_date_from_today
import numpy as np
import pandas as pd
import cufflinks as cf
import datetime as dt
import plotly.plotly as pt
import time
from ta import *
from plotly.offline import init_notebook_mode, iplot

## Load the "REFINITV WORKSPACE" API
To use the "Refinitiv Workspace" API, you will first need to generate a key within Refinitiv Workspace. 

Open the APPKEY app in the Worksapce and generate an App Key. Make sure the status is set to "Enabled"

In [2]:
ek.set_app_key('1ef16e8281d849d19a52fb738238da3e76465d28')   #Insert your App Key here in quotations

## Inputs

Enter your desired RICS in a list.

Your start date will be inputed as "x days ago" using the get_date_from_today function. Enter your desired x value in the parenthesis.

Additionally, you will enter your desired calculation period for the Simple Moving Average in the variable SMA, as well as the period setting for RSI Analysis as well.

In [3]:
rics = ["DIS.N","AAPL.O","GOOGL.O","AMZN.O"] 

now = dt.date.today()
start_dt = get_date_from_today(360)  #Change the value in the parenthesis to your desired start date. 
                                     #In this example, our start date will be 360 days before today.

SMA = 20
RSI_period = 14

## Use the API to Extract Close Price data for desired time period

In [4]:
data = ek.get_timeseries(rics,
                         fields = "CLOSE",
                         start_date = start_dt,       #If you want to use a time period that does not end today
                         end_date = "{}".format(now)) #you can manually input the dates.

#Use help(ek.get_timeseries) if you receive an error message

### Creating copies of extracted data for each analysis
We will create copies of the data we extracted using the API to use with Bollinger Band Indicators, RSI Indicators, and MACD Indicators

In [5]:
data_B = data.copy()
data_R = data.copy()
data_M = data.copy()

# Bollinger Band Indicator

Bollinger Bands are constructed based on the standard deviation of closing prices over the last n periods. An analyst creates high and low bands usually two standard deviations above and below the  n-period moving average. The bands move away from one another when price volatility increases and move closer together when prices are less volatile.

Bollinger bands are viewed as useful for indicating when prices are extreme by recent standards on either the high or low side. Prices at or above the upper Bollinger band may be viewed as indicating an overbought market, one that is "too high" and likely to decrease in the near term. Prices at or below the lower Bollinger band may be viewed as indicating an oversold market, one that is "too low" and likely to increase in the near term. 

### Calculating Simple Moving Average, Upper and Lower Bands, and Indicators of Oversold/OverBought Markets 

add_SMA_bands will calculate a simple moving average of n periods as specified in the inputs. It will additionally create the Upper and Lower Bollinger bands to the dataframe, as well as in indicator "OVER" that has a value of 1 if the price > upper Bollinger band or price < lower Bollinger band.

In [6]:
def add_SMA_bands(data,ric):
    df = pd.DataFrame(data[ric])
    SMAname = "SMA_{}".format(ric)
    SMAstd = "SMAstd_{}".format(ric)
    SMAUpper = "UpperBand_{}".format(ric)
    SMALower = "LowerBand_{}".format(ric)
    df[SMAname] = df["{}".format(ric)].rolling(SMA).mean()   #Simple Moving Average of Close Price  
    df[SMAUpper] = df[SMAname] + (df["{}".format(ric)].rolling(SMA).std())*2    #Upper Band that is 2 STD above moving avg
    df[SMALower] = df[SMAname] - (df["{}".format(ric)].rolling(SMA).std())*2    #Lower Band that is 2 STD below moving avg
    df["OVER"] = np.where(df[ric]<df["LowerBand_{}".format(ric)],-1,
        (np.where(df[ric]>df["UpperBand_{}".format(ric)],1,0)))
    return df

Using the function created above, we will create a dictionary of dataframes (one for each RIC you have gotten pricing information for). This process will be repeated for the other tools.

In [7]:
dfs_B={}

for ric in rics:
    df = add_SMA_bands(data_B,ric) #this is a function, just adds other columns to the dataframe
    df.dropna(inplace=True)
    df["OVER"][df["OVER"]==0] = np.nan
    dfs_B[ric]=df

From the dictionary of dataframes, you can graph the Bollinger Band indicator for a specific RIC by changing the RIC in the quotes.

Ex. dfs_B["______"] <---- INSERT RIC HERE INSIDE QUOTES

In [20]:
dfs_B["DIS.N"].iplot(kind="lines",secondary_y = "OVER",yTitle="Price",xTitle="Date", title="Bollinger Band Indicators")

# Oscillators

Oscillators are indicators based on market prices but scaled so that they "oscillate" around a given value, such as zero (MACD Oscillator), or between two values (RSI). Extreme high values of an oscillator are viewed as indicating that a market is overbought, while extreme low values are viewed as indicating an oversold market. 

We will use 2 oscillators here: The Relative Strength Index and Moving Average Convergence Divergence

# Relative Strength Index

An RSI is based on the ratio of total price increases to total price decreases over a selected number of periods, selected in the inputs above. This ratio is then scaled to oscillate between 0 and 100.

High values (typically over 70) indicate an overbought market, and low values (typically below 30) indicate an oversold market.

The function get_RSI will split price changes to increases to decreases and calculate RSI and indicators for overbought and oversold markets are created. Again, a dictionary of DataFrames is created for each RIC. 

In [21]:
def get_RSI(series,period,ric):
    df_r = pd.DataFrame(series)
    delta = df_r.diff().dropna()
    up_days = delta.copy()
    up_days[delta<=0] = 0.0
    down_days=abs(delta.copy())
    down_days[delta>0] = 0.0
    RS_up = up_days.rolling(period).mean()
    RS_down = down_days.rolling(period).mean()
    RSI = 100 - 100/(1+RS_up/RS_down)
    return RSI

In [22]:
dfs_R={}

for ric in rics:
    df_r = get_RSI(data_R[ric],RSI_period,ric)
    df_r['OVERSOLD'] = 30
    df_r['OVERBOUGHT'] = 70
    df_r['OVERBOUGHT_INDIC'] = np.where(df_r[ric]>df_r['OVERBOUGHT'],1,0)
    df_r['OVERSOLD_INDIC'] = np.where(df_r[ric]<df_r['OVERSOLD'],1,0)
    df_r["OVERBOUGHT_INDIC"][df_r["OVERBOUGHT_INDIC"]==0] = np.nan
    df_r["OVERSOLD_INDIC"][df_r["OVERSOLD_INDIC"]==0] = np.nan
    dfs_R[ric] = df_r

#This section creates the shaded area below
htrial = {}

htrial["y0"] = "30"
htrial["y1"] = "70"
htrial["color"] = 'rgba(30,30,30,0.3)'
htrial["fill"] = True
htrial["opacity"] = .4

In [23]:
#To view the RSI of another RIC, change the RIC in the quotes
dfs_R["DIS.N"].iplot(kind="lines", hspan = htrial, yTitle="RSI",xTitle="Date", title="Relative Strength Index Indicators") 

# Moving Average Convergence Divergence Indicator

MACD Oscillators are created using exponentially smoothed moving averages, which place a greater weight on more recent observations. The MACD line is the difference between two exponentially smoothed moving averages of the price, and the "signal line" is an exponentially smoothed moving average of the MACD line. The lines osciallte around zero but are not bounded.

For the MACD line, a 12-period and 26-period EMA is typically used, and for the signal line, a 9-period EMA of the MACD is commonly used.

Points where the two lines cross can be used as trading signals. When the MACD line crosses ABOVE the signal line, this indicates an oversold market. When the MACD crosses BELOW the signal line, this indicates an overbought market. 

The MACD_calc function creates the exponential moving averages used in MACD. The signals created when the MACD line crosses the signal line is not visable on the plot, but if you hover the plot you will be able to see the indicators at the intersects.

In [39]:
def MACD_calc(data,ric):
    df_m = pd.DataFrame(data)
    df_m["26_EMA"] = df_m[ric].ewm(span=26).mean()
    df_m["12_EMA"] = df_m[ric].ewm(span=12).mean()
    df_m['MACD'] = (df_m["12_EMA"] - df_m["26_EMA"])
    df_m["Signal_Line"] = df_m["MACD"].ewm(span=9).mean()
    df_m["Zero"] = 0.0
    return df_m

In [40]:
dfs_M = {}

for ric in rics:
    df_m = MACD_calc(data_M[ric],ric)
    df_m = df_m.drop(columns = [ric,"26_EMA","12_EMA"])
    df_m["OVERBOUGHT"] = np.where((df_m["MACD"] < df_m["Signal_Line"]) & (df_m["MACD"].shift(1) > df_m["Signal_Line"].shift(1)),1,0)
    df_m["OVERSOLD"] = np.where((df_m["MACD"] > df_m["Signal_Line"]) & (df_m["MACD"].shift(1) < df_m["Signal_Line"].shift(1)),1,0)
    df_m["OVERBOUGHT"][df_m["OVERBOUGHT"]==0] = np.nan
    df_m["OVERSOLD"][df_m["OVERSOLD"]==0] = np.nan    
    dfs_M[ric] = df_m

In [41]:
dfs_M["DIS.N"].iplot(kind="lines",yTitle="MACD",xTitle="Date", title="Moving Average Convergence Divergence Indicators")

# Visualizing All Indicators

In [42]:
data["DIS.N"].iplot(kind='lines',yTitle='Price',xTitle='Date',title='Close Price')
dfs_B["DIS.N"].iplot(kind="lines",secondary_y = "OVER",yTitle="Price",xTitle="Date", title="Bollinger Band Indicators")
dfs_R["DIS.N"].iplot(kind="lines", hspan = htrial, yTitle="RSI",xTitle="Date", title="Relative Strength Index Indicators")
dfs_M["DIS.N"].iplot(kind="lines",yTitle="MACD",xTitle="Date", title="Moving Average Convergence Divergence Indicators")

# Creating Buy and Sell Signals

Now that 3 price based and oscillator indicators to create signals of an overbought and oversold market have been created, buy and sell signals can be created. 

If a majority (2/3) of the models indicate that the market is Overbought or Oversold, a Buy or Sell recommendation will be created.

indicator_function will take the signals for overbought/sold markets into a separate dataframe, creates a majority vote to generate signals.

In [43]:
def indicator_function(bollingerIndicator,RSIindicator,MACDIndicator):
    df_i = bollingerIndicator[['OVER']].copy()
    df_i["OVER"] = abs(df_i["OVER"])
    df_i["RSIBought"] = RSIindicator[['OVERBOUGHT_INDIC']].copy()
    df_i["RSISold"] = RSIindicator[['OVERSOLD_INDIC']].copy()
    df_i["MACDBought"] = MACDIndicator[["OVERBOUGHT"]].copy()
    df_i["MACDSold"] = MACDIndicator[["OVERSOLD"]].copy()
    df_i["SellNum"] = np.nansum([df_i["OVER"], df_i["RSIBought"], df_i["MACDBought"]], axis=0)
    df_i["BuyNum"] = np.nansum([df_i["OVER"] , df_i["RSISold"] ,df_i["MACDSold"]], axis=0)
    df_i["SellSignal"] = np.where(df_i["SellNum"]>1,"SELL",0)
    df_i["BuySignal"] = np.where(df_i["BuyNum"]>1,"BUY",0)
    df_i = df_i.drop(columns = ["OVER","RSIBought","RSISold","MACDBought","MACDSold","BuyNum","SellNum"])
    return df_i

In [44]:
indicator = {}

for ric in rics:
    df_i = indicator_function(dfs_B[ric],dfs_R[ric],dfs_M[ric])
    indicator[ric] = df_i

To view recommendations for a particular stock, just change the ric in quotations

In [45]:
indicator["DIS.N"]

,SellSignal,BuySignal
Date,,
2018-12-13,0,0
2018-12-14,0,0
2018-12-17,0,0
2018-12-18,0,0
2018-12-19,0,0
2018-12-20,0,BUY
2018-12-21,0,BUY
2018-12-24,0,BUY
2018-12-26,0,0


To view the most recent recommendations of a stock, use the line below. Change the value in the parenthesis in .tail(x) to to control how many data points are generated

In [55]:
indicator["DIS.N"].tail(5)

,SellSignal,BuySignal
Date,,
2019-11-04,0,0
2019-11-05,0,0
2019-11-06,0,0
2019-11-07,0,0
2019-11-08,SELL,0
